# 1. Classifying LinkedIn Posts as Story-based or Not

- What is included
- how it was obtained
- important details about how it was sampled


# 2. Data Conversion

In [7]:
import pandas as pd

df = pd.read_csv("./posts.csv")
df[df["engagement"] == 0]


,content,engagement,comments
13,3 Offline Marketing Lessons That Will Boost Yo...,0,0
38,🎙️ I'm excited to share that I've finally reco...,0,0
59,We would love to invite you and your friends t...,0,0
61,I've read a ton of books.\n\nCould I give you ...,0,0
98,"[EDIT UPDATE 9/11] Okay, this is it. I need he...",0,0
122,"I don't know about you, but I'm feeling that e...",0,0
123,How do you build long-term wealth?,0,0
180,"And BairesDev became orange! 🧡\n\nThis week, o...",0,0
181,We like to sip coffee or tea from the warmth a...,0,0
189,Charging based on your output instead of an ho...,0,0


# 3. Data Cleaning, Pre-Processing, and Feature Engineering

# 4. The Task
- classification
- Need to split into training and test data sets

# 5. Model Selection

# 6. Model Training

Students will need to select a sample of the digital data they have accumulated in their life so far. This might include images, sound files, text, geospatial, or numerical data. This can include the video, text, and other data created by their participation in this and other Minerva classes. The first section**** of the notebook should explain the data: what is included, how it was obtained, and all important details about how it was sampled from the student's own digital archive.

The second section**** of the notebook should contain well-commented code for converting this data to python readable format (and scikit-learn) and loading this data into an appropriate data structure (np.array, pandas dataframe, glob etc.).

The third section of the notebook**** should include a markdown section explaining any necessary cleaning, pre-processing, and feature engineering the data requires, and a include a code block completing these steps. You may also wish to perform some basic exploratory data analysis at this point reporting and visualizing some samples and appropriate descriptive statistics.

The fourth section**** should include a markdown section discussing the task (classification, regression, or clustering) that will be conducted on the data, along with well commented code block that splits data into training and test sets.

The fifth section**** should discuss model**** selection in a markdown section and include model initialization and construction in a well-commented code block. This section should include a clear discussion of the model's mathematical underpinnings. You might include the training algorithm as pseudocode and/or an explanation of the loss function in markdown.

The sixth section_**** _should train the model, including any necessary cross validation and hyperparameter tuning.

The seventh section**** should contain code to generate predictions for out of sample data, and compute appropriate performance**** metrics.

The eighth section_ _should visualize the results and discuss your conclusions.

The ninth section should be an executive summary of the prior eight sections, clearly explaining your steps, diagramming your pipeline, visualizing any key results, and explaining any key insights or shortcomings of your approach. You may wish to include a discussion of how the model might be improved.
The tenth section should contain references for documents, guides, or code repos you accessed for the project.

# 7. Prediction & Performance Evaluation

# 8. Result Visualization

# 9. Executive Summary

# 10. References